In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
zone = spark.read.csv("/Volumes/ingestao/default/ingestao_dados/taxi_zone_lookup.csv", header=True)
yellowTripData01 = spark.read.parquet("/Volumes/ingestao/default/ingestao_dados/yellow_tripdata_2025-01.parquet")
yellowTripData02 = spark.read.parquet("/Volumes/ingestao/default/ingestao_dados/yellow_tripdata_2025-02.parquet")
yellowTripData03 = spark.read.parquet("/Volumes/ingestao/default/ingestao_dados/yellow_tripdata_2025-03.parquet")
yellowTripData04 = spark.read.parquet("/Volumes/ingestao/default/ingestao_dados/yellow_tripdata_2025-04.parquet")
yellowTripData05 = spark.read.parquet("/Volumes/ingestao/default/ingestao_dados/yellow_tripdata_2025-05.parquet")
yellowTripData06 = spark.read.parquet("/Volumes/ingestao/default/ingestao_dados/yellow_tripdata_2025-06.parquet")
yellowTripData07 = spark.read.parquet("/Volumes/ingestao/default/ingestao_dados/yellow_tripdata_2025-07.parquet")
yellowTripData08 = spark.read.parquet("/Volumes/ingestao/default/ingestao_dados/yellow_tripdata_2025-08.parquet")
yellowTripData09 = spark.read.parquet("/Volumes/ingestao/default/ingestao_dados/yellow_tripdata_2025-09.parquet")

In [0]:
display(yellowTripData09.head(5))

In [0]:
import re
def to_camel_case(name):
    if not name:
        return name

    name = re.sub(r"^tpep_", "", name)

    if "_" in name:
        parts = name.lower().split("_")
        return parts[0] + "".join(p.capitalize() for p in parts[1:])

    name = re.sub(r'([A-Z]+)([A-Z][a-z])', r'\1_\2', name)
    name = re.sub(r'([a-z])([A-Z])', r'\1_\2', name)

    parts = name.lower().split("_")
    camel = parts[0] + "".join(p.capitalize() for p in parts[1:])
    return camel



def rename_columns_to_camel_case(df):
    for old_col in df.columns:
        new_col = to_camel_case(old_col)
        df = df.withColumnRenamed(old_col, new_col)
    return df

In [0]:
yellowTrips = (
    yellowTripData01
    .union(yellowTripData02)
    .union(yellowTripData03)
    .union(yellowTripData04)
    .union(yellowTripData05)
    .union(yellowTripData06)
    .union(yellowTripData07)
    .union(yellowTripData08)
    .union(yellowTripData09)
)


In [0]:
display(rename_columns_to_camel_case(yellowTrips))

In [0]:
yellowTrips = rename_columns_to_camel_case(yellowTrips) 
zone = rename_columns_to_camel_case(zone)

In [0]:
yellowTrips.printSchema()

In [0]:
zone.printSchema()

In [0]:
yellowTrips = (
    yellowTrips
    .withColumn("pickupDatetime", col("pickupDatetime").cast(TimestampType()))
    .withColumn("dropoffDatetime", col("dropoffDatetime").cast(TimestampType()))
    
    .withColumn("vendorId", col("vendorId").cast(IntegerType()))
    .withColumn("passengerCount", col("passengerCount").cast(IntegerType()))
    .withColumn("rateCodeId", col("rateCodeId").cast(IntegerType()))
    .withColumn("paymentType", col("paymentType").cast(IntegerType()))
    .withColumn("puLocationId", col("puLocationId").cast(IntegerType()))
    .withColumn("doLocationId", col("doLocationId").cast(IntegerType()))
    
    .withColumn("tripDistance", col("tripDistance").cast(DoubleType()))
    .withColumn("fareAmount", col("fareAmount").cast(DoubleType()))
    .withColumn("extra", col("extra").cast(DoubleType()))
    .withColumn("mtaTax", col("mtaTax").cast(DoubleType()))
    .withColumn("tipAmount", col("tipAmount").cast(DoubleType()))
    .withColumn("tollsAmount", col("tollsAmount").cast(DoubleType()))
    .withColumn("improvementSurcharge", col("improvementSurcharge").cast(DoubleType()))
    .withColumn("totalAmount", col("totalAmount").cast(DoubleType()))
    .withColumn("congestionSurcharge", col("congestionSurcharge").cast(DoubleType()))
    .withColumn("airportFee", col("airportFee").cast(DoubleType()))
    .withColumn("cbdCongestionFee", col("cbdCongestionFee").cast(DoubleType()))
    
    .withColumn("storeAndFwdFlag", col("storeAndFwdFlag").cast(StringType()))
)

zone = zone.withColumn("LocationID", col("LocationID").cast(IntegerType())) \
           .withColumn("Borough", col("Borough").cast(StringType())) \
           .withColumn("Zone", col("Zone").cast(StringType()))


In [0]:
yellowTrips.printSchema()

In [0]:
@dlt.table(
    name="bronze.default.yellow_taxi_tripdata_2025",
)
def yellow_taxi_bronze():
    return yellowTrips

@dlt.table(
    name="bronze.default.taxi_zone_lookup",
)
def taxi_zone_bronze():
    return zone